In [10]:
import sqlite3

import pandas as pd

pd.options.plotting.backend = "plotly"

In [11]:
conexao = sqlite3.connect('../mydb.sqlite')
cursor = conexao.cursor()

try:
    # Executar uma consulta
    cursor.execute("SELECT * FROM expressions")

    # Buscar todos os resultados
    resultados = cursor.fetchall()

    # Cria um dataframe a partir dos resultados
    df = pd.DataFrame(resultados, columns=['timestamp', 'neutral', 'happy', 'sad', 'angry', 'fearful', 'disgusted', 'surprised'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', origin='unix')
    df.set_index('timestamp', inplace=True)
    df = df[df.index.notnull()]
    df = df[~df.index.duplicated()]
        
    # Arredonda os valores para 4 casas decimais exceto da timestamp
    df = df.round(3)

    # Se o valor for menor que 0.5, ele passa a ser null
    df[df < 0.01] = None

    print(df)

except sqlite3.Error as e:
    print("Erro ao executar a consulta:", e)

finally:
    # Verificar se o cursor está aberto antes de tentar fechá-lo
    if cursor:
        cursor.close()

    # Fechar a conexão
    conexao.close()

                               neutral  happy  sad  angry  fearful  disgusted  \
timestamp                                                                       
2024-01-28 15:06:26.668999936    1.000    NaN  NaN    NaN      NaN        NaN   
2024-01-28 15:06:26.684999936    1.000    NaN  NaN    NaN      NaN        NaN   
2024-01-28 15:06:26.772999936    1.000    NaN  NaN    NaN      NaN        NaN   
2024-01-28 15:06:26.784000000    1.000    NaN  NaN    NaN      NaN        NaN   
2024-01-28 15:06:26.912999936    1.000    NaN  NaN    NaN      NaN        NaN   
...                                ...    ...  ...    ...      ...        ...   
2024-01-28 15:17:13.251000064    0.963  0.037  NaN    NaN      NaN        NaN   
2024-01-28 15:17:14.256000000    0.484  0.515  NaN    NaN      NaN        NaN   
2024-01-28 15:17:17.251000064    0.960  0.040  NaN    NaN      NaN        NaN   
2024-01-28 15:17:18.257999872    0.998    NaN  NaN    NaN      NaN        NaN   
2024-01-28 15:17:21.25100006

In [12]:
import plotly.graph_objects as go
import pandas as pd

colors = ["#997600", "#e83326", "#008c5c", "#1077f3", "#c85b00", "#bf8cfc", "#9b54f3", "#ee74ee", "#5ba8f7", "#339933", "#444488"]

# Salva o DataFrame como csv
df.to_csv('expressions.csv')
print('expressions.csv salvo com sucesso!')

# Lê o csv novamente para um novo DataFrame
data = pd.read_csv("expressions.csv")
data['timestamp'] = pd.to_datetime(data['timestamp'])  # Converta a coluna timestamp para datetime se não estiver

fig = go.Figure()

data = data.set_index('timestamp')
# data = data.resample('5s').mean()  # Resample para obter médias em intervalos de 5 segundos

traces = [
    go.Scattergl(
        x=data.index,
        y=data[col].astype(float),
        name=col, mode='markers+lines',
        marker={'size': 2},
    ) for col in data.columns if col != 'timestamp'
]

fig.add_traces(traces)
fig.update_layout(hovermode='x', colorway=colors)
fig.update_traces(hovertemplate='%{y}')
fig.write_html('expressions.html')


expressions.csv salvo com sucesso!
